# WEEK 3: DATA WAREHOUSE AND BIGQUERY

## 3.1 Data Warhouse

### - OLAP vs OLTP

- OLAP -> Online Analytical Processing

- OLTP -> Online Transactional Processing

|       | OLTP | OLAP |
| ----------- | ----------- |----------- |
| Purpose | Control and run essential business operations in real life | Plan, solve problems, support decisions and discover hidden insights |
| Data update | Shirt, fast updates initiated by users | Data periodically refreshed with scheduled, long running batch job |
| Database design | Normalized databases for effeciency | Denormalized databases for analysis |
| Space requirements | Generally small if historical data is archived | Generally large due to aggregating large dataset |
| Backup and recovery | Regular backups required to ensure business continuity and meet legal and governence requirements | Lost data can be reloaded from OLTP database as needed in lieu of reguler backups |
| Productivity | Increase productivity of end users | Increase productivity of business managers, data analysts and executives |
| Data view | Lists day-to-day business transactions | Multi dimensional view of enterprise data |
| User examples | Customer-facing personel, clerks, online shoppers | Knowledge workers, data analysts, business analyst and executives |


### - Data Warehouse

- OLAP solution

- Used for reporting and data analysis

![Alt text](images/Data%20warehouse.png)

### - BigQuery

- Serverless data warehouse
- Software as well as infrastructure including scalability and high availability
- Built-in features
  - Machine learning
  - Geospatial analysis
  - Bussiness intelligence
- BigQuery maximizes flexibility by separating the compute engine that analyze data from storage

### - BigQuery Cost

- On demand pricing
  - 1 TB of data processed = $5
- Flat rate pricing
  - Based on number of pre requested slots
  - 100 slots -> $2000/month = 400 TB data processed on demand pricing





